In [1]:
import re
import string
import joblib
import contractions
import pandas as pd
from unidecode import unidecode

# NLP & Spell Correction
from symspellpy import SymSpell
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# spelling correct  assets 

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

sym_spell.load_dictionary('../artifacts/frequency_dictionary_en_82_765.txt', term_index=0, count_index=1)

def correct_spelling(text):
    corrected = sym_spell.lookup_compound(text, max_edit_distance=2)
    return corrected[0].term if corrected else text

In [3]:
# stop words assets

with open('../artifacts/stop.txt', 'r', encoding = 'utf-8') as file:
    stopWords = file.readlines()

stopWords = [word.replace('\n', '') for word in stopWords]

In [4]:
# normalize text assets

def normalize_text(text):
    text = contractions.fix(text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    text = unidecode(text)  
    return text


In [5]:
# load the count vectorizer assets
def split_into_words(i):
    return [word for word in i.split(" ")]

preprocessedBow = joblib.load('../model/preprocessedBow')

In [6]:
# Data preprocessing 

def DataPreprocessing(textData):

    # lower conversion
    lowerConversion = textData.lower()

    # Spelling correction
    spellingCorrection = correct_spelling(lowerConversion)

    #Punctual removal 
    punctual_removal = spellingCorrection.translate(str.maketrans('', '', string.punctuation+'\n'))

    # stop words removal
    stopWordsRemoval = " ".join([words for words in punctual_removal.split() if words not in stopWords])

    # normalize text 
    normalizeText = normalize_text(stopWordsRemoval)

    return normalizeText

In [18]:
testData = """
   The movie was not bad at all.
"""
preprocessedText = DataPreprocessing(testData)

In [19]:
preprocessedText

'movie bad'

In [20]:
TestData = preprocessedBow.transform([preprocessedText])

In [21]:
print(TestData)

  (0, 2244)	1
  (0, 21430)	1


# model 

In [11]:
model = joblib.load('../model/sentiment.joblib')

In [22]:
model.predict(TestData)

array(['negative'], dtype='<U8')